In [36]:
import ipaddress
import random
import jinja2

net1 = ipaddress.ip_network(u'192.168.0.0/16')
net2 = ipaddress.ip_network(u'172.16.0.0/16')

alist1 = list(net1)
alist2 = list(net2)

random.shuffle(alist1)
random.shuffle(alist2)

In [37]:
acl_raw = []
for n, (src, dst) in enumerate(zip(alist1, alist2)):
    acl_raw.append((n, src, dst))

In [38]:
HOST = '127.0.0.1'
PORT = 8305
USER = 'cisco'
PASS = 'cisco'

In [47]:
from ncclient import manager
from lxml import etree

def pretty_print(nc_retval):
        print(etree.tostring(etree.fromstring(nc_retval.data_xml), pretty_print=True))
        
def my_unknown_host_cb(host, fingerprint):
    return True
    
m = manager.connect(host=HOST, port=PORT, username=USER, password=PASS,
                    timeout=86400,
                    allow_agent=False,
                    look_for_keys=False,
                    hostkey_verify=False,
                    unknown_host_cb=my_unknown_host_cb)

In [48]:
ACL_TEMPLATE = jinja2.Template('''<config>
  <ipv4-acl-and-prefix-list xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ipv4-acl-cfg">
   <accesses>
    <access>
     <access-list-name>{{ACL_NAME}}</access-list-name>
     <access-list-entries>
      {% for N, A, B in LIST %}<access-list-entry>
       <sequence-number>{{N}}</sequence-number>
       <grant>permit</grant>
       <source-network>
        <source-address>{{A}}</source-address>
       </source-network>
       <destination-network>
        <destination-address>{{B}}</destination-address>
       </destination-network>
       <sequence-str>10</sequence-str>{% endfor %}
      </access-list-entry>
     </access-list-entries>
    </access>
   </accesses>
  </ipv4-acl-and-prefix-list>
 </config>''')

In [49]:
big_acl = ACL_TEMPLATE.render(ACL_NAME='BIG', LIST=acl_raw)

In [50]:
print(len(big_acl))

23862754


In [46]:
try:
    m.edit_config(big_acl, target='candidate', format='xml')
    m.commit()
    c = m.get_config(source='running', filter=('subtree', filter))
    pretty_print(c)
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

NameError: name 'TimeoutExpiredError' is not defined

In [31]:
filter = '''<ipv4-acl-and-prefix-list xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ipv4-acl-cfg"/>'''

c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <ipv4-acl-and-prefix-list xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ipv4-acl-cfg">
   <accesses>
    <access>
     <access-list-name>BIG</access-list-name>
     <access-list-entries>
      <access-list-entry>
       <sequence-number>10</sequence-number>
       <grant>permit</grant>
       <source-network>
        <source-address>192.168.91.174</source-address>
       </source-network>
       <destination-network>
        <destination-address>172.16.22.46</destination-address>
       </destination-network>
       <sequence-str>10</sequence-str>
      </access-list-entry>
     </access-list-entries>
    </access>
   </accesses>
  </ipv4-acl-and-prefix-list>
 </data>

